<a href="https://colab.research.google.com/github/itsmeeeeeee/MML/blob/main/Sk_learn_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Majority Voting Ensemble Method with SK-Learn framework

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch

# Load labels from CSV
file_path_labels = "/content/drive/MyDrive/MultimodalNLP/projekt/data/labels.csv"
labeled_data = pd.read_csv(file_path_labels)
labels = labeled_data['overall_sentiment'].values

# Define your existing mapping
label_mapping = {
    "very_negative": 0,
    "negative": 0,
    "positive": 1,
    "very_positive": 1
}

# Get valid indices for labels that exist in the mapping
valid_indices = [i for i, label in enumerate(labels) if label in label_mapping]
filtered_labels = [labels[i] for i in valid_indices]

# Apply the mapping to convert filtered text labels to numeric labels
numeric_labels = np.array([label_mapping[label] for label in filtered_labels])

# Convert labels to a torch tensor
numeric_labels = torch.tensor(numeric_labels, dtype=torch.long)
print("numeric labels:", numeric_labels.shape)

# Load image features
file_path_im = "/content/drive/MyDrive/MultimodalNLP/projekt/features_data/image_features_restnet.npy"
image_features = np.load(file_path_im)

# Load text features
file_path_emb = "/content/drive/MyDrive/MultimodalNLP/projekt/features_data/text_features_bert.npy"
text_features = np.load(file_path_emb)

# Filter image and text features using valid indices
filtered_image_features = torch.tensor(image_features[valid_indices], dtype=torch.float32)
filtered_text_features = torch.tensor(text_features[valid_indices], dtype=torch.float32)

print("Filtered Image features:", filtered_image_features.shape)
print("Filtered Text features:", filtered_text_features.shape)


numeric labels: torch.Size([4791])
Filtered Image features: torch.Size([4791, 1000])
Filtered Text features: torch.Size([4791, 768])


In [5]:


# Split the data into training and testing sets


train_text, test_text, train_image, test_image, train_labels, test_labels = train_test_split(
    filtered_text_features, filtered_image_features, numeric_labels, test_size=0.2, random_state=42)  #20%


# Combine the text and image features into a single tensor
train_features = np.hstack((train_text.numpy(), train_image.numpy()))
test_features = np.hstack((test_text.numpy(), test_image.numpy()))
train_labels = train_labels.numpy()
test_labels = test_labels.numpy()



## **Train the models**
- Decision Tree Classifier
- Logistic Regression
- Multilayer Perceptron
- Adaptive Boosting
- Support Vector Machine


Implemention of K-Fold cross-validator.
[K-Fold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)


In [7]:
# split dhe data with Kfold method and train all 5 models
kf = KFold(n_splits=5, shuffle=True, random_state=42)
tree_models = []
adaboost_models = []
svm_models = []
logreg_models = []
mlp_models =[]

for train_idx, val_idx in kf.split(train_features):
    # split the data for K-Fold
    X_train, X_val = train_features[train_idx], train_features[val_idx]
    y_train, y_val = train_labels[train_idx], train_labels[val_idx]

    # Decision Tree Classifier
    tree = DecisionTreeClassifier(random_state=42)
    tree.fit(X_train, y_train)
    tree_models.append(tree)

    # AdaBoost Classifier
    adaboost = AdaBoostClassifier(random_state=42)
    adaboost.fit(X_train, y_train)
    adaboost_models.append(adaboost)

     # SVM Classifier
    svm = SVC(probability=True, random_state=42)  # probability=True für Voting='soft'
    svm.fit(X_train, y_train)
    svm_models.append(svm)

   # Logistic Regression
    logreg = LogisticRegression(max_iter=200, random_state=42)
    logreg.fit(X_train, y_train)
    logreg_models.append(logreg)

  #MLP Classifier
    mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=200, activation='relu', solver='adam', random_state=42)
    mlp.fit(X_train, y_train)
    mlp_models.append(mlp)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

### Performance of Decision Tree Classifier


In [10]:
predictions =  tree.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)


Test Accuracy: 0.7445255474452555
Test F1 Score: 0.4892249297278026


### Performance of MLP Classifier

In [11]:
predictions =  mlp.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)


Test Accuracy: 0.8112617309697602
Test F1 Score: 0.5014175128179351


### Performance of  AdaBoost

In [12]:
predictions =  adaboost.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)



Test Accuracy: 0.8738269030239834
Test F1 Score: 0.49718997664452447


### Performance of SVM

In [13]:
predictions =  svm.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)

Test Accuracy: 0.8873826903023984
Test F1 Score: 0.47016574585635357


### Performance of Logistic Regression

In [14]:
predictions =  logreg.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)

Test Accuracy: 0.8477580813347236
Test F1 Score: 0.4964974108170311


## Voting Method

In [15]:
# Voting Classifier initialisieren

#[('adaboost' + str(i), adaboost) for i, adaboost in enumerate(adaboost_models)]
#    [('svm' + str(i), svm) for i, svm in enumerate(svm_models)] +
   # [('logreg' + str(i), logreg)for i,logreg in enumerate(logreg_models)]+
voting_clf = VotingClassifier(
    estimators=[('tree' + str(i), tree) for i, tree in enumerate(tree_models)] +
    [('adaboost' + str(i), adaboost) for i, adaboost in enumerate(adaboost_models)] +
    [('logreg' + str(i), logreg)for i,logreg in enumerate(logreg_models)]+
    [('svm' + str(i), svm) for i, svm in enumerate(svm_models)] +

    [('mlp' + str(i), mlp)for i,mlp in enumerate(mlp_models)],
  # voting='soft' #acc: 0.53, f1 macro 0.30

    voting='soft'
)
voting_clf.fit(train_features, train_labels)  # Notwendig, um das Schema der Classifiers abzugleichen

# Testdaten bewerten
predictions = voting_clf.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='macro')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Test Accuracy: 0.872784150156413
Test F1 Score: 0.4740622864746287


In [17]:

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)

Test Accuracy: 0.872784150156413
Test F1 Score: 0.4740622864746287
